In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
user_actions = pd.read_csv('user_actions.csv', dtype={'message': str},
    parse_dates=['createdDateTime'])

In [3]:
test = user_actions.sort_values(['actionId'])[:20]
test

,actionId,currentUser,createdDateTime,message
8653,0007077a-5d0f-49a4-91d6-0b8dc32837c6,sxj,2019-12-23 11:33:00,[START] [IMPORT]: [num_records=523]
14075,0007077a-5d0f-49a4-91d6-0b8dc32837c6,sxj,2019-12-23 11:39:24,"[END] [IMPORT]: [num_success=246, num_failed=2..."
6634,000b03f4-afd8-4371-b79e-2891d5388181,lfcjx,2019-12-23 14:11:02,"[END] [UPDATE]: [num_success=898, num_failed=2..."
10945,000b03f4-afd8-4371-b79e-2891d5388181,lfcjx,2019-12-23 14:11:00,[START] [UPDATE]: [num_records=986]
11517,000b147d-1d2e-41a6-8ab6-d8888c50e12a,ozraghkiejdns,2019-07-28 09:04:00,[START] [APPROVE]: [num_records=453]
4939,000b147d-1d2e-41a6-8ab6-d8888c50e12a,ozraghkiejdns,2019-07-28 09:06:37,"[END] [APPROVE]: [num_success=145, num_failed=..."
4944,002e3d10-33e4-45aa-b882-0d8fab4c5819,lfcjx,2019-03-18 15:40:00,[START] [APPROVE]: [num_records=531]
12869,002e3d10-33e4-45aa-b882-0d8fab4c5819,lfcjx,2019-03-18 15:47:54,"[END] [APPROVE]: [num_success=378, num_failed=..."
18020,0031390e-202f-447f-9de0-044851d7e78a,gkxqjitrflwmuay,2019-09-27 16:10:02,"[END] [IMPORT]: [num_success=222, num_failed=1..."
7162,0031390e-202f-447f-9de0-044851d7e78a,gkxqjitrflwmuay,2019-09-27 16:10:00,[START] [IMPORT]: [num_records=499]


In [4]:
def parse_msg(msg):
    string_regex = re.findall(r'\[(.*?)\]', msg)
    return string_regex

In [5]:
def new_table(base, type_msg):
    drop = base[base['message'].apply(lambda x: not x.startswith(type_msg))].index
    new_table = base.drop(drop).loc[:,['actionId', 'createdDateTime', 'message']]
    new_table['action'] = new_table.apply(lambda x: parse_msg(x['message'])[1], axis=1)
    new_table['msg'] = new_table.apply(lambda x: parse_msg(x['message'])[2], axis=1)
    new_table.drop(columns='message', inplace=True)
#     new_table = new_table.set_index(['actionId'])
    return new_table

In [6]:
test_start = new_table(test, '[START]')
test_start

,actionId,createdDateTime,action,msg
8653,0007077a-5d0f-49a4-91d6-0b8dc32837c6,2019-12-23 11:33:00,IMPORT,num_records=523
10945,000b03f4-afd8-4371-b79e-2891d5388181,2019-12-23 14:11:00,UPDATE,num_records=986
11517,000b147d-1d2e-41a6-8ab6-d8888c50e12a,2019-07-28 09:04:00,APPROVE,num_records=453
4944,002e3d10-33e4-45aa-b882-0d8fab4c5819,2019-03-18 15:40:00,APPROVE,num_records=531
7162,0031390e-202f-447f-9de0-044851d7e78a,2019-09-27 16:10:00,IMPORT,num_records=499
18559,00328b5c-cfc8-4606-a826-4f12439054c4,2019-05-20 08:01:00,IMPORT,num_records=221
13004,0038e7bc-24a2-4f21-a819-8fc6bed6f268,2019-07-20 10:24:00,IMPORT,num_records=435
10106,00395294-d240-4861-a2d8-28da413612b3,2019-07-15 12:18:00,SEARCH,approved_by=bikz
3343,004ac46f-775d-4260-a0a4-9be55fec7adb,2019-10-07 11:32:00,IMPORT,num_records=707
16665,004d3774-5293-491e-84f3-0381195ac133,2019-08-27 10:04:00,UPDATE,num_records=503


In [7]:
test_end = new_table(test, '[END]')
test_end

,actionId,createdDateTime,action,msg
14075,0007077a-5d0f-49a4-91d6-0b8dc32837c6,2019-12-23 11:39:24,IMPORT,"num_success=246, num_failed=25, num_warning=252"
6634,000b03f4-afd8-4371-b79e-2891d5388181,2019-12-23 14:11:02,UPDATE,"num_success=898, num_failed=29, num_warning=59"
4939,000b147d-1d2e-41a6-8ab6-d8888c50e12a,2019-07-28 09:06:37,APPROVE,"num_success=145, num_failed=213, num_warning=95"
12869,002e3d10-33e4-45aa-b882-0d8fab4c5819,2019-03-18 15:47:54,APPROVE,"num_success=378, num_failed=116, num_warning=37"
18020,0031390e-202f-447f-9de0-044851d7e78a,2019-09-27 16:10:02,IMPORT,"num_success=222, num_failed=194, num_warning=83"
9109,00328b5c-cfc8-4606-a826-4f12439054c4,2019-05-20 08:01:19,IMPORT,"num_success=135, num_failed=64, num_warning=22"
1627,0038e7bc-24a2-4f21-a819-8fc6bed6f268,2019-07-20 10:24:11,IMPORT,"num_success=302, num_failed=82, num_warning=51"
12387,00395294-d240-4861-a2d8-28da413612b3,2019-07-15 12:18:08,SEARCH,num_results=259
1311,004ac46f-775d-4260-a0a4-9be55fec7adb,2019-10-07 11:33:47,IMPORT,"num_success=323, num_failed=91, num_warning=293"
17225,004d3774-5293-491e-84f3-0381195ac133,2019-08-27 10:08:08,UPDATE,"num_success=344, num_failed=16, num_warning=143"


In [8]:
test_join = test_start.set_index(['actionId', 'action'])\
    .join(test_end.set_index(['actionId', 'action']), 
          lsuffix='_start', 
          rsuffix='_end',
          how='inner')

In [9]:
test_join['delta'] = test_join.apply(lambda x: (x['createdDateTime_end'] - x['createdDateTime_start']), axis = 1)
test_join

,,createdDateTime_start,msg_start,createdDateTime_end,msg_end,delta
actionId,action,,,,,
0007077a-5d0f-49a4-91d6-0b8dc32837c6,IMPORT,2019-12-23 11:33:00,num_records=523,2019-12-23 11:39:24,"num_success=246, num_failed=25, num_warning=252",00:06:24
000b03f4-afd8-4371-b79e-2891d5388181,UPDATE,2019-12-23 14:11:00,num_records=986,2019-12-23 14:11:02,"num_success=898, num_failed=29, num_warning=59",00:00:02
000b147d-1d2e-41a6-8ab6-d8888c50e12a,APPROVE,2019-07-28 09:04:00,num_records=453,2019-07-28 09:06:37,"num_success=145, num_failed=213, num_warning=95",00:02:37
002e3d10-33e4-45aa-b882-0d8fab4c5819,APPROVE,2019-03-18 15:40:00,num_records=531,2019-03-18 15:47:54,"num_success=378, num_failed=116, num_warning=37",00:07:54
0031390e-202f-447f-9de0-044851d7e78a,IMPORT,2019-09-27 16:10:00,num_records=499,2019-09-27 16:10:02,"num_success=222, num_failed=194, num_warning=83",00:00:02
00328b5c-cfc8-4606-a826-4f12439054c4,IMPORT,2019-05-20 08:01:00,num_records=221,2019-05-20 08:01:19,"num_success=135, num_failed=64, num_warning=22",00:00:19
0038e7bc-24a2-4f21-a819-8fc6bed6f268,IMPORT,2019-07-20 10:24:00,num_records=435,2019-07-20 10:24:11,"num_success=302, num_failed=82, num_warning=51",00:00:11
00395294-d240-4861-a2d8-28da413612b3,SEARCH,2019-07-15 12:18:00,approved_by=bikz,2019-07-15 12:18:08,num_results=259,00:00:08
004ac46f-775d-4260-a0a4-9be55fec7adb,IMPORT,2019-10-07 11:32:00,num_records=707,2019-10-07 11:33:47,"num_success=323, num_failed=91, num_warning=293",00:01:47


# Retrieving Data That Cannot Be Processed

In [10]:
test_join[test_join.isnull().any(axis=1)]

,,createdDateTime_start,msg_start,createdDateTime_end,msg_end,delta
actionId,action,,,,,


In [11]:
test_join[test_join.isnull().any(axis=1)].count()

createdDateTime_start    0
msg_start                0
createdDateTime_end      0
msg_end                  0
delta                    0
dtype: int64

# Percentage of operations (pie chart)

In [ ]:
test.abs